In [31]:
import pandas as pd
import re
import nltk
import spacy
from num2words import num2words
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import pos_tag
from spacy.pipeline import EntityRecognizer

In [32]:
# Read the JSON file into a DataFrame
df = pd.read_json('Resume.json', lines=True)

# Save the DataFrame to CSV
df.to_csv('dataframe.csv', index=None)

# Print the shape of the dataset
print(f"Shape of the dataset: {df.shape}")

# Display the first few rows of the DataFrame
df.head()

Shape of the dataset: (200, 2)


,content,annotation
0,Govardhana K\nSenior Software Engineer\n\nBeng...,"[{'label': ['Companies worked at'], 'points': ..."
1,"Harini Komaravelli\nTest Analyst at Oracle, Hy...","[{'label': ['Companies worked at'], 'points': ..."
2,Hartej Kathuria\nData Analyst Intern - Oracle ...,"[{'label': ['Skills'], 'points': [{'start': 22..."
3,Ijas Nizamuddin\nAssociate Consultant - State ...,"[{'label': ['Skills'], 'points': [{'start': 46..."
4,"Imgeeyaul Ansari\njava developer\n\nPune, Maha...","[{'label': ['Skills'], 'points': [{'start': 18..."


In [33]:
df['content'][0]

'Govardhana K\nSenior Software Engineer\n\nBengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/\nb2de315d95905b68\n\nTotal IT experience 5 Years 6 Months\nCloud Lending Solutions INC 4 Month • Salesforce Developer\nOracle 5 Years 2 Month • Core Java Developer\nLanguages Core Java, Go Lang\nOracle PL-SQL programming,\nSales Force Developer with APEX.\n\nDesignations & Promotions\n\nWilling to relocate: Anywhere\n\nWORK EXPERIENCE\n\nSenior Software Engineer\n\nCloud Lending Solutions -  Bangalore, Karnataka -\n\nJanuary 2018 to Present\n\nPresent\n\nSenior Consultant\n\nOracle -  Bangalore, Karnataka -\n\nNovember 2016 to December 2017\n\nStaff Consultant\n\nOracle -  Bangalore, Karnataka -\n\nJanuary 2014 to October 2016\n\nAssociate Consultant\n\nOracle -  Bangalore, Karnataka -\n\nNovember 2012 to December 2013\n\nEDUCATION\n\nB.E in Computer Science Engineering\n\nAdithya Institute of Technology -  Tamil Nadu\n\nSeptember 2008 to June 2012\n\nhttps://www.

In [34]:
df['annotation'][0]

[{'label': ['Companies worked at'],
  'points': [{'start': 1749, 'end': 1754, 'text': 'Oracle'}]},
 {'label': ['Companies worked at'],
  'points': [{'start': 1696, 'end': 1701, 'text': 'Oracle'}]},
 {'label': ['Companies worked at'],
  'points': [{'start': 1417, 'end': 1422, 'text': 'Oracle'}]},
 {'label': ['Skills'],
  'points': [{'start': 1356,
    'end': 1792,
    'text': 'Languages: Core Java, Go Lang, Data Structures & Algorithms, Oracle\nPL-SQL programming, Sales Force with APEX.\nTools: RADTool, Jdeveloper, NetBeans, Eclipse, SQL developer,\nPL/SQL Developer, WinSCP, Putty\nWeb Technologies: JavaScript, XML, HTML, Webservice\n\nOperating Systems: Linux, Windows\nVersion control system SVN & Git-Hub\nDatabases: Oracle\nMiddleware: Web logic, OC4J\nProduct FLEXCUBE: Oracle FLEXCUBE Versions 10.x, 11.x and 12.x'}]},
 {'label': ['Companies worked at'],
  'points': [{'start': 1209, 'end': 1214, 'text': 'Oracle'}]},
 {'label': ['Skills'],
  'points': [{'start': 1136,
    'end': 1247,


## Data Cleaning

### Text Cleaning

In [35]:
def clean_text(resume):
    # Convert the text to lowercase
    resume = resume.lower()
    
    # Remove newlines
    resume = re.sub("\n", ' ', resume)
    
    # Remove special characters
    resume = re.sub(r'[,•()➢❑]', ' ', resume)
    
    # Remove extra whitespaces, dashes, and dots
    resume = re.sub(r'\s\s+|\s-\s|\.\s', ' ', resume)
    
    # Tokenize the text into words
    tokenized_words = resume.split(" ")
    
    length = len(tokenized_words)
    
    # Convert digits to words
    for i in range(length):
        if tokenized_words[i].isdigit():
            tokenized_words[i] = num2words(tokenized_words[i])
        
    # Remove stopwords
    sw = set(stopwords.words('english'))
    tokens_without_sw = []
    for w in tokenized_words:
        if w not in sw:
            tokens_without_sw.append(w)
    
    # Join the tokens back into a string
    final_resume = " ".join(tokens_without_sw)
    
    return final_resume


### Clean content column

In [37]:
content_resumes = df['content']

for i in range(0,200):
    content_resumes[i] = clean_text(content_resumes[i])

In [38]:
df['content'][0]

'govardhana k senior software engineer bengaluru karnataka karnataka email indeed: indeed.com/r/govardhana-k/ b2de315d95905b68 total experience five years six months cloud lending solutions inc four month salesforce developer oracle five years two month core java developer languages core java go lang oracle pl-sql programming sales force developer apex  designations & promotions willing relocate: anywhere work experience senior software engineer cloud lending solutions  bangalore karnataka  january two thousand and eighteen present present senior consultant oracle  bangalore karnataka  november two thousand and sixteen december two thousand and seventeen staff consultant oracle  bangalore karnataka  january two thousand and fourteen october two thousand and sixteen associate consultant oracle  bangalore karnataka  november two thousand and twelve december two thousand and thirteen education b.e computer science engineering adithya institute technology  tamil nadu september two thousand

In [39]:
df['content'][1]

"harini komaravelli test analyst oracle hyderabad hyderabad telangana email indeed: indeed.com/r/harini- komaravelli/2659eee82e435d1b six yrs experience manual automation testing  work experience qa analyst oracle test analyst oracle hyderabad infosys ltd  hyderabad telangana  november two thousand and eleven february two thousand and sixteen hyderabad nov two thousand and eleven feb17 two thousand and sixteen worked tata consultancy services hyderabad feb twenty-four apr eleven two thousand and seventeen currently working test analyst oracle hyderabad qa analyst six years experience oracle education mca osmania university b.sc computer science osmania university skills functional testing blue prism qtp additional information area expertise: familiar agile methodologies  knowledge energy petroleum & health care domains  involved preparation test scenarios  preparing test data test cases  https://www.indeed.com/r/harini-komaravelli/2659eee82e435d1b?isid=rex-download&ikw=download-top&co=

### Word Analysis

In [40]:
# We use the Tf-Idf vectorizer to gain insights about the probabilities of all the possible words and collocations
# we can find in these resumes.

# Create an instance of the Tf-Idf vectorizer with a specified ngram range
vect = TfidfVectorizer(ngram_range=(1, 3))

# Apply the Tf-Idf vectorizer to the 'content_resumes' data
tf_idf = vect.fit_transform(content_resumes)

# Retrieve the terms (words and collocations) in the same order as they appear in the tf_idf matrix
terms = vect.get_feature_names_out()

# Print the tf_idf matrix
print(tf_idf)


  (0, 344)	0.0392661003166396
  (0, 272)	0.0392661003166396
  (0, 137)	0.0392661003166396
  (0, 118959)	0.0392661003166396
  (0, 43688)	0.0392661003166396
  (0, 74443)	0.0392661003166396
  (0, 43683)	0.0392661003166396
  (0, 82323)	0.0392661003166396
  (0, 72530)	0.0392661003166396
  (0, 62382)	0.0392661003166396
  (0, 120304)	0.0392661003166396
  (0, 67734)	0.0392661003166396
  (0, 74524)	0.0392661003166396
  (0, 28603)	0.0392661003166396
  (0, 50074)	0.0392661003166396
  (0, 46406)	0.0392661003166396
  (0, 105593)	0.0392661003166396
  (0, 106190)	0.0392661003166396
  (0, 24520)	0.0392661003166396
  (0, 118928)	0.036428206199717955
  (0, 121189)	0.036428206199717955
  (0, 61877)	0.036428206199717955
  (0, 106376)	0.03285288244566607
  (0, 73835)	0.02873890004625888
  (0, 120480)	0.0392661003166396
  :	:
  (199, 53218)	0.008534517639336575
  (199, 3294)	0.008942340037136233
  (199, 123414)	0.01204278283231081
  (199, 73208)	0.010115007446032302
  (199, 99480)	0.007442812760637008
  (19

In [41]:
terms
 

array(['000', '000 servers', '000 servers trouble', ..., 'zxf05u01',
       'zxf05u01 validating', 'zxf05u01 validating vendor'], dtype=object)

In [42]:
### Showing the part of speech that these terms belong to

nltk.pos_tag(terms)

[('000', 'CD'),
 ('000 servers', 'NNS'),
 ('000 servers trouble', 'CD'),
 ('000members', 'NNS'),
 ('000members evaluatedpatientcareneeds', 'CD'),
 ('000members evaluatedpatientcareneeds prioritizedtreatment', 'CD'),
 ('0023411a049a1441', 'CD'),
 ('0023411a049a1441 challenging', 'VBG'),
 ('0023411a049a1441 challenging career', 'CD'),
 ('0023411a049a1441 isid', 'CD'),
 ('0023411a049a1441 isid rex', 'CD'),
 ('005e1ab800b4cb42', 'CD'),
 ('005e1ab800b4cb42 isid', 'CD'),
 ('005e1ab800b4cb42 isid rex', 'CD'),
 ('005e1ab800b4cb42 work', 'CD'),
 ('005e1ab800b4cb42 work experience', 'CD'),
 ('00f125c7b9b95a35', 'CD'),
 ('00f125c7b9b95a35 isid', 'CD'),
 ('00f125c7b9b95a35 isid rex', 'CD'),
 ('00f125c7b9b95a35 two', 'CD'),
 ('00f125c7b9b95a35 two year', 'CD'),
 ('01', 'CD'),
 ('01 crm', 'CD'),
 ('01 crm 02', 'CD'),
 ('01 dopra', 'CD'),
 ('01 dopra description', 'CD'),
 ('01 ethernet', 'CD'),
 ('01 ethernet routing', 'VBG'),
 ('01 tnc', 'CD'),
 ('01 tnc controller', 'CD'),
 ('01st', 'CD'),
 ('01st 

In [43]:
### Displaying the features table, where columns are the possible mono-, bi- and tri-grams in all of the resumes.
### We try to identify the most common words and collocations to use in our NER model, later.

pd.DataFrame.sparse.from_spmatrix(tf_idf, index = content_resumes, columns=terms)[0:2]

000  000 servers  \
content                                                                
govardhana k senior software engineer bengaluru...  0.0          0.0   
harini komaravelli test analyst oracle hyderaba...  0.0          0.0   

                                                    000 servers trouble  \
content                                                                   
govardhana k senior software engineer bengaluru...                  0.0   
harini komaravelli test analyst oracle hyderaba...                  0.0   

                                                    000members  \
content                                                          
govardhana k senior software engineer bengaluru...         0.0   
harini komaravelli test analyst oracle hyderaba...         0.0   

                                                    000members evaluatedpatientcareneeds  \
content                                                                                    
govardhana k senior software engineer bengaluru...                                   0.0   
harini komaravelli test analyst oracle hyderaba...                                   0.0   

                                                    000members evaluatedpatientcareneeds prioritizedtreatment  \
content                                                                                                         
govardhana k senior software engineer bengaluru...                                                0.0           
harini komaravelli test analyst oracle hyderaba...                                                0.0           

                                                    0023411a049a1441  \
content                                                                
govardhana k senior software engineer bengaluru...               0.0   
harini komaravelli test analyst oracle hyderaba...               0.0   

                                                    0023411a049a1441 challenging  \
content                                                                            
govardhana k senior software engineer bengaluru...                           0.0   
harini komaravelli test analyst oracle hyderaba...                           0.0   

                                                    0023411a049a1441 challenging career  \
content                                                                                   
govardhana k senior software engineer bengaluru...                                  0.0   
harini komaravelli test analyst oracle hyderaba...                                  0.0   

                                                    0023411a049a1441 isid  \
content                                                                     
govardhana k senior software engineer bengaluru...                    0.0   
harini komaravelli test analyst oracle hyderaba...                    0.0   

                                                    ...  zoom  zoom knowledge  \
content                                             ...                         
govardhana k senior software engineer bengaluru...  ...   0.0             0.0   
harini komaravelli test analyst oracle hyderaba...  ...   0.0             0.0   

                                                    zoom knowledge webrtc  \
content                                                                     
govardhana k senior software engineer bengaluru...                    0.0   
harini komaravelli test analyst oracle hyderaba...                    0.0   

                                                    zoom supporting  \
content                                                               
govardhana k senior software engineer bengaluru...              0.0   
harini komaravelli test analyst oracle hyderaba...              0.0   

                                                    zoom supporting bada2  \
content                                                                     
go

In [44]:
### This specific resume is causing a great deal of problems because of the issue of no spaces (delimiters) between the words.

content_resumes[154]

"jacob philip kottayam kerala email indeed: indeed.com/r/jacob-philip/db00d831146c9228 strategicsales experienceinsales skills.currently success  work experience sales marketing specialist assistantbusinessdevelopmentmanager  dubai ae  february two thousand and seventeen october two thousand and seventeen uae -builtstrong clientrelationshipsandprovidedhighvalue-addingservices resultingina15% company marketshareincrease developstools practicesacrosstheorganization negotiatingcontractsandpackages negotiatingthetermsofanagreementwithaviewto closingsale expense andnew businessdata workedcloselywithpartners throughconductingqualityassurancetests.actasthepointofcontactandcommunicate projectstatustoallparticipantsinourteam ordinator marketingco  january two thousand and fifteen june two thousand and sixteen bhimajewelers kerala india systemreportforms plannedandexecutedeventsandmarketingprograms producingfivetimestargetnumberof qualifiedleads implements marketing advertising campaigns assembl

In [45]:
### Final Clean Resume Sample

print(content_resumes[1])

harini komaravelli test analyst oracle hyderabad hyderabad telangana email indeed: indeed.com/r/harini- komaravelli/2659eee82e435d1b six yrs experience manual automation testing  work experience qa analyst oracle test analyst oracle hyderabad infosys ltd  hyderabad telangana  november two thousand and eleven february two thousand and sixteen hyderabad nov two thousand and eleven feb17 two thousand and sixteen worked tata consultancy services hyderabad feb twenty-four apr eleven two thousand and seventeen currently working test analyst oracle hyderabad qa analyst six years experience oracle education mca osmania university b.sc computer science osmania university skills functional testing blue prism qtp additional information area expertise: familiar agile methodologies  knowledge energy petroleum & health care domains  involved preparation test scenarios  preparing test data test cases  https://www.indeed.com/r/harini-komaravelli/2659eee82e435d1b?isid=rex-download&ikw=download-top&co=i

In [46]:
### NLTK with word tokenization results in seperating urls

tokenized_words = word_tokenize(content_resumes[1])

nltk.pos_tag(tokenized_words)

[('harini', 'NN'),
 ('komaravelli', 'JJ'),
 ('test', 'NN'),
 ('analyst', 'NN'),
 ('oracle', 'NN'),
 ('hyderabad', 'NN'),
 ('hyderabad', 'NN'),
 ('telangana', 'NN'),
 ('email', 'VBP'),
 ('indeed', 'RB'),
 (':', ':'),
 ('indeed.com/r/harini-', 'JJ'),
 ('komaravelli/2659eee82e435d1b', 'NN'),
 ('six', 'CD'),
 ('yrs', 'NN'),
 ('experience', 'NN'),
 ('manual', 'JJ'),
 ('automation', 'NN'),
 ('testing', 'VBG'),
 ('work', 'NN'),
 ('experience', 'NN'),
 ('qa', 'JJ'),
 ('analyst', 'NN'),
 ('oracle', 'NN'),
 ('test', 'NN'),
 ('analyst', 'NN'),
 ('oracle', 'NN'),
 ('hyderabad', 'NN'),
 ('infosys', 'NN'),
 ('ltd', 'NN'),
 ('hyderabad', 'NN'),
 ('telangana', 'JJ'),
 ('november', 'RB'),
 ('two', 'CD'),
 ('thousand', 'NN'),
 ('and', 'CC'),
 ('eleven', 'RB'),
 ('february', 'JJ'),
 ('two', 'CD'),
 ('thousand', 'NN'),
 ('and', 'CC'),
 ('sixteen', 'JJ'),
 ('hyderabad', 'NN'),
 ('nov', 'RB'),
 ('two', 'CD'),
 ('thousand', 'NN'),
 ('and', 'CC'),
 ('eleven', 'RB'),
 ('feb17', 'JJ'),
 ('two', 'CD'),
 ('thousa

### Word Tagging

In [48]:
def tag_words(text):
    # Load the English language model in spaCy
    nlp = spacy.load("en_core_web_sm")
    
    # Process the text with the language model
    words = nlp(text)
    
    tagged = []
    
    # Iterate over each word in the processed text
    for word in words:
        # Print the word and its part-of-speech tag
        print(word, word.pos_)
        
        # Append the word and its part-of-speech tag to the tagged list
        tagged.append((word.text, word.pos_))
        
    return tagged

In [50]:
### Using spacy tagging is prone to errors

tag_words(content_resumes[1])

harini PROPN
komaravelli PROPN
test PROPN
analyst PROPN
oracle PROPN
hyderabad PROPN
hyderabad PROPN
telangana PROPN
email NOUN
indeed ADV
: PUNCT
indeed.com/r/harini- PROPN
komaravelli/2659eee82e435d1b PROPN
six NUM
yrs NOUN
experience NOUN
manual ADJ
automation NOUN
testing VERB
  SPACE
work NOUN
experience NOUN
qa NOUN
analyst NOUN
oracle PROPN
test PROPN
analyst NOUN
oracle PROPN
hyderabad PROPN
infosys PROPN
ltd PROPN
  SPACE
hyderabad PROPN
telangana PROPN
  SPACE
november PROPN
two NUM
thousand NUM
and CCONJ
eleven NUM
february NOUN
two NUM
thousand NUM
and CCONJ
sixteen NUM
hyderabad NOUN
nov PROPN
two NUM
thousand NUM
and CCONJ
eleven NUM
feb17 NUM
two NUM
thousand NUM
and CCONJ
sixteen NUM
worked VERB
tata NOUN
consultancy NOUN
services NOUN
hyderabad PROPN
feb PROPN
twenty PROPN
- PUNCT
four NUM
apr NOUN
eleven NUM
two NUM
thousand NUM
and CCONJ
seventeen NUM
currently ADV
working VERB
test NOUN
analyst NOUN
oracle PROPN
hyderabad PROPN
qa PROPN
analyst NOUN
six NUM
years NO

[('harini', 'PROPN'),
 ('komaravelli', 'PROPN'),
 ('test', 'PROPN'),
 ('analyst', 'PROPN'),
 ('oracle', 'PROPN'),
 ('hyderabad', 'PROPN'),
 ('hyderabad', 'PROPN'),
 ('telangana', 'PROPN'),
 ('email', 'NOUN'),
 ('indeed', 'ADV'),
 (':', 'PUNCT'),
 ('indeed.com/r/harini-', 'PROPN'),
 ('komaravelli/2659eee82e435d1b', 'PROPN'),
 ('six', 'NUM'),
 ('yrs', 'NOUN'),
 ('experience', 'NOUN'),
 ('manual', 'ADJ'),
 ('automation', 'NOUN'),
 ('testing', 'VERB'),
 (' ', 'SPACE'),
 ('work', 'NOUN'),
 ('experience', 'NOUN'),
 ('qa', 'NOUN'),
 ('analyst', 'NOUN'),
 ('oracle', 'PROPN'),
 ('test', 'PROPN'),
 ('analyst', 'NOUN'),
 ('oracle', 'PROPN'),
 ('hyderabad', 'PROPN'),
 ('infosys', 'PROPN'),
 ('ltd', 'PROPN'),
 (' ', 'SPACE'),
 ('hyderabad', 'PROPN'),
 ('telangana', 'PROPN'),
 (' ', 'SPACE'),
 ('november', 'PROPN'),
 ('two', 'NUM'),
 ('thousand', 'NUM'),
 ('and', 'CCONJ'),
 ('eleven', 'NUM'),
 ('february', 'NOUN'),
 ('two', 'NUM'),
 ('thousand', 'NUM'),
 ('and', 'CCONJ'),
 ('sixteen', 'NUM'),
 ('hyd